In [17]:
import pandas as pd
import numpy as np

df = pd.read_excel('data.xlsx')

In [18]:
df.head(1)

,序号,来源详情,您宝宝的性别,您的宝宝的出生日期：,(1)您有几个孩子,(2)个孩子，这是您的第___个孩子,宝宝的主要抚养人,请问宝宝的家庭类型是,宝宝母亲的年龄（周岁）,宝宝母亲的文化程度:,...,拥有足量的与年龄适宜的玩具,对孩子任何事情都要干涉,给孩子的诉说、表达或申辩的机会,带孩子到商店、公园,赞赏孩子的好奇心，有意识培养孩子兴趣,无端向孩子发脾气,有意识帮助孩子认知事物，扩大词汇,保持孩子活动的房间清洁、整齐,对孩子的需求能够作出及时的回应.1,总分
0,1,NaN,男,2023-06-20,1,2,保姆阿姨,联合家庭（宝宝、父母、祖父母（或外祖父母）三代一起居住）,23,初中及以下,...,(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),62


In [19]:
df.shape

(52640, 142)

In [20]:
df.columns

Index(['序号', '来源详情', '您宝宝的性别', '您的宝宝的出生日期：', '(1)您有几个孩子', '(2)个孩子，这是您的第___个孩子',
       '宝宝的主要抚养人', '请问宝宝的家庭类型是', '宝宝母亲的年龄（周岁）', '宝宝母亲的文化程度:',
       ...
       '拥有足量的与年龄适宜的玩具', '对孩子任何事情都要干涉', '给孩子的诉说、表达或申辩的机会', '带孩子到商店、公园',
       '赞赏孩子的好奇心，有意识培养孩子兴趣', '无端向孩子发脾气', '有意识帮助孩子认知事物，扩大词汇', '保持孩子活动的房间清洁、整齐',
       '对孩子的需求能够作出及时的回应.1', '总分'],
      dtype='object', length=142)

## 数据预处理

In [21]:
print('总实例数', df.shape[0])
nullValue = df.isnull().sum()
for i in range(len(nullValue)):
    if nullValue[i] != 0 :
        print('缺失特征名:', df.columns[i], '\n缺失:', nullValue[i])

总实例数 52640
缺失特征名: 来源详情 
缺失: 43121


缺失值填充，`来源详情` 的 `NAN` 填充为 `0`

In [22]:
df.fillna(0, axis=1, inplace=True)
df.isnull().any()

序号                   False
来源详情                 False
您宝宝的性别               False
您的宝宝的出生日期：           False
(1)您有几个孩子            False
                     ...  
无端向孩子发脾气             False
有意识帮助孩子认知事物，扩大词汇     False
保持孩子活动的房间清洁、整齐       False
对孩子的需求能够作出及时的回应.1    False
总分                   False
Length: 142, dtype: bool

序号和出生日期是无用的，删去这两列

分割目标列，总分

In [23]:
df_drop = df.drop(columns=['序号', '您的宝宝的出生日期：'], axis=1)

data = df_drop.drop('总分', axis=1)
target = df_drop['总分']

feature = data.columns

输出每一个问题的答案种类到文件 `value.txt`

In [24]:
def saveUnique(df, feature, fileName):
    file = open(fileName, 'w+')
    for i in feature:
        print(df[i].unique(), file=file)
    file.close()

def lookUnique(df, feature):
    for i in feature:
        print(df[i].unique())


saveUnique(data, feature, 'value.txt')

In [25]:
decoder = lambda a : dict(zip( a, range(len(a)) ))
dicList = [
    decoder([0, '直接访问']),
    decoder(['男', '女']),
    {'1':1, '2':2, '3':3, '一':1, '二':2, '三':3, '俩':2, '两':2, '两个':2, '一个':1, '4':4,
     '四':4, '5':5, '五':5, '三个':3, '四个':4, '五个':5}, # (1)您有几个孩子
    {'1':1, '2':2, '3':3, '一':1, '二':2, '三':3, '俩':2, '两':2, '两个':2, '一个':1, '4':4,
     '四':4, '5':5, '五':5, '三个':3, '四个':4, '五个':5, '第一':1, '第一个':1, '第二':2, '第二个':2}, # (2)个孩子，这是您的第___个孩子
    {'保姆阿姨':0, '父母':1, '外祖父母或祖父母':2}, # 宝宝的主要抚养人
    decoder(['联合家庭（宝宝、父母、祖父母（或外祖父母）三代一起居住）','核心家庭（宝宝与父母一起居住）','其他（单亲、再婚等）']), # 请问宝宝的家庭类型是
    {}, # 宝宝母亲的年龄（周岁） 
    decoder(['初中及以下','本科','大专','高中及中专','研究生','博士生']), # 宝宝母亲的文化程度:
    {}, # 宝宝父亲的年龄（周岁）
    decoder(['初中及以下','本科','大专','高中及中专','研究生','博士生']), # 宝宝父亲文化的程度:
    decoder(['很好','一般','比较差','比较好','很差']), # 最近一年，您家庭经济情况如何:
    decoder(['县级市', '涉农区', '城区']), # 您宝宝本次体检的社区属于什么区？
    decoder(['0~1岁', '3~6岁', '1~3岁']), # 您的宝宝年龄
]

len(dicList) # 来源详情 + 基本问题 = 1 + 12
for i in range(13):
    data.replace({feature[i]:dicList[i]}, inplace=True)

In [26]:
saveUnique(data, feature, "valueA.txt")

In [27]:
opstr = lambda x: 3 if isinstance(x, str) and x.startswith('其他') else (x if isinstance(x, int) else -1)
data[feature[4]] = data[feature[4]].apply(opstr)

In [28]:
opnum = lambda x: int(x) if isinstance(x, str) and len(x) <= 2 and x.isdigit() else (-1 if isinstance(x, str) else x)
for i in [6, 8]:
    data[feature[i]] = data[feature[i]].apply(opnum)
saveUnique(data, feature, "valueB.txt")

In [29]:
opinvaild = lambda x: -1 if isinstance(x, str) else x
for i in [2, 3]:
    data[feature[i]] = data[feature[i]].apply(opinvaild)
saveUnique(data, feature, "valueC.txt")

In [33]:
mask = ~data.isin([-1]).any(axis=1)
data = data[mask]
data.head(3)

,来源详情,您宝宝的性别,(1)您有几个孩子,(2)个孩子，这是您的第___个孩子,宝宝的主要抚养人,请问宝宝的家庭类型是,宝宝母亲的年龄（周岁）,宝宝母亲的文化程度:,宝宝父亲的年龄（周岁）,宝宝父亲文化的程度:,...,经常强迫孩子多吃一点,拥有足量的与年龄适宜的玩具,对孩子任何事情都要干涉,给孩子的诉说、表达或申辩的机会,带孩子到商店、公园,赞赏孩子的好奇心，有意识培养孩子兴趣,无端向孩子发脾气,有意识帮助孩子认知事物，扩大词汇,保持孩子活动的房间清洁、整齐,对孩子的需求能够作出及时的回应.1
0,0,0,1,2,0,0,23,0,36,3,...,(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过)
1,0,0,2,2,1,1,30,1,31,3,...,很少,总是,总是,总是,总是,总是,很少,总是,总是,总是
2,0,1,2,2,1,0,30,2,34,2,...,(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过)


In [34]:
saveUnique(data, feature, 'result.txt')

In [35]:
data.shape

(52010, 139)

处理问题列

In [36]:
f = open('tmp', 'w+')
for i in feature:
    print(i, file=f)
f.close()